Data - https://www.kaggle.com/kiranraje/prediction-facebook-comment

### Предобработка датасета

1. Выберите подходящую метрику качества (можно взять несколько)  
2. Сделайте предобработку признаков для линейной регрессии

In [1]:
val DATA_PATH = "data/Dataset.csv"

DATA_PATH: String = "data/Dataset.csv"

In [2]:
import $ivy.`org.scalanlp::breeze:1.0`
import $ivy.`org.plotly-scala::plotly-almond:0.7.6`

import $ivy.$                         

import $ivy.$                                      

In [5]:
import breeze.linalg._
import breeze.numerics._
import java.io.{FileReader}
import breeze.io.{CSVReader,CSVWriter}
import breeze.numerics._
import breeze.stats.regression.{leastSquares, lasso}
import plotly._, plotly.element._, plotly.layout._, plotly.Almond._

import breeze.linalg._

import breeze.numerics._

import java.io.{FileReader}

import breeze.io.{CSVReader,CSVWriter}

import breeze.numerics._

import breeze.stats.regression.{leastSquares, lasso}

import plotly._, plotly.element._, plotly.layout._, plotly.Almond._

Чтение csv файла - пропущенные значения заполняются 0

In [6]:
def csvreadWithNA(reader: java.io.Reader,
                  separator: Char = ',',
                  quote: Char = '\"',
                  escape: Char = '\\',
                  skipLines: Int = 0): 
    DenseMatrix[Double] = {
        var mat = CSVReader.read(reader, separator, quote, escape, skipLines)
        mat = mat.takeWhile(line => line.length != 0 && line.head.nonEmpty) // empty lines at the end
    reader.close()
    if (mat.length == 0) {
        DenseMatrix.zeros[Double](0, 0)
    } else {
        DenseMatrix.tabulate(mat.length, mat.head.length)((i, j) => if(mat(i)(j).length == 0) 0 else mat(i)(j).toDouble)
    }
}

defined function csvreadWithNA

In [7]:
val reader = new FileReader(DATA_PATH)
val matrix=csvreadWithNA(reader, separator=',', escape = '\\', skipLines=1)

reader: FileReader = java.io.FileReader@22ff3d2c
matrix: DenseMatrix[Double] = 634995.0  0.0  463.0  1.0  0.0   0.0   0.0   0.0   0.0    65.0  ... (28 total)
634995.0  0.0  463.0  1.0  0.0   0.0   0.0   0.0   0.0    10.0  ...
634995.0  0.0  463.0  1.0  0.0   0.0   0.0   0.0   0.0    14.0  ...
634995.0  0.0  463.0  1.0  7.0   0.0   3.0   7.0   -3.0   62.0  ...
634995.0  0.0  463.0  1.0  1.0   0.0   0.0   1.0   0.0    58.0  ...
634995.0  0.0  463.0  1.0  0.0   0.0   0.0   0.0   0.0    60.0  ...
634995.0  0.0  463.0  1.0  0.0   0.0   0.0   0.0   0.0    68.0  ...
634995.0  0.0  463.0  1.0  1.0   0.0   1.0   1.0   -1.0   32.0  ...
634995.0  0.0  463.0  1.0  0.0   0.0   0.0   0.0   0.0    35.0  ...
634995.0  0.0  463.0  1.0  0.0   0.0   0.0   0.0   0.0    48.0  ...
634995.0  0.0  463.0  1.0  0.0   0.0   0.0   0.0   0.0    52.0  ...
634995.0  0.0  463.0  1.0  1.0   0.0   0.0   1.0   0.0    69.0  ...
634995.0  0.0  463.0  1.0  0.0   0.0   0.0   0.0   0.0    3.0   ...
634995.0  0.0  463.0  1.0 

In [8]:
println("Dim: " + matrix.rows + " " + matrix.cols)

Dim: 40949 28


В качестве метрики можно взять RMSE

### Моделирование
1. Постройте модель простой линейной регрессии, используя стандартные функции Breeze. Оцените качество модели.

In [9]:
val y = matrix(::, 27)
val X = matrix(::, 0 to 26)
println("Dim: " + X.rows + " " + X.cols)

Dim: 40949 27


y: DenseVector[Double] = DenseVector(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 15.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 1.0, 0.0, 0.0, 3.0, 0.0, 1.0, 0.0, 0.0, 3.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 2.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 3.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 3.0, 0.0, 342.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 35.0, 92.0, 51.0, 119.0, 2.0, 7.0, 71.0, 14.0, 8.0, 30.0, 2.0, 3.0, 4.0, 10.0, 7.0, 409.0, 28.0, 372.0, 23.0, 102.0, 26.0, 431.0, 4.0, 1.0, 6.0, 145.0, 7.0, 4.0, 90.0, 760.0, 57.0, 501.0, 6.0, 15.0, 56.0, 31.0, 642.0, 43.0, 320.0, 55.0, 7.0, 128.0, 233.0, 20.0, 53.0, 53.0, 280.0, 200.0, 94

Разбиваем данные на train и test. 

In [10]:
val TRAIN_SIZE = X.rows * 0.8

val X_train = X(0 to TRAIN_SIZE.toInt,::)
val X_test = X(TRAIN_SIZE.toInt to X.rows-1, ::)

val y_train = y(0 to TRAIN_SIZE.toInt)
val y_test = y(TRAIN_SIZE.toInt to X.rows-1)

TRAIN_SIZE: Double = 32759.2
X_train: DenseMatrix[Double] = 634995.0  0.0  463.0  1.0  0.0   0.0   0.0   0.0   0.0    65.0  ... (27 total)
634995.0  0.0  463.0  1.0  0.0   0.0   0.0   0.0   0.0    10.0  ...
634995.0  0.0  463.0  1.0  0.0   0.0   0.0   0.0   0.0    14.0  ...
634995.0  0.0  463.0  1.0  7.0   0.0   3.0   7.0   -3.0   62.0  ...
634995.0  0.0  463.0  1.0  1.0   0.0   0.0   1.0   0.0    58.0  ...
634995.0  0.0  463.0  1.0  0.0   0.0   0.0   0.0   0.0    60.0  ...
634995.0  0.0  463.0  1.0  0.0   0.0   0.0   0.0   0.0    68.0  ...
634995.0  0.0  463.0  1.0  1.0   0.0   1.0   1.0   -1.0   32.0  ...
634995.0  0.0  463.0  1.0  0.0   0.0   0.0   0.0   0.0    35.0  ...
634995.0  0.0  463.0  1.0  0.0   0.0   0.0   0.0   0.0    48.0  ...
634995.0  0.0  463.0  1.0  0.0   0.0   0.0   0.0   0.0    52.0  ...
634995.0  0.0  463.0  1.0  1.0   0.0   0.0   1.0   0.0    69.0  ...
634995.0  0.0  463.0  1.0  0.0   0.0   0.0   0.0   0.0    3.0   ...
634995.0  0.0  463.0  1.0  1.0   1.0   0.0   

In [11]:
val lr = leastSquares(X_train, y_train)

Aug 23, 2020 6:16:16 PM com.github.fommil.netlib.LAPACK <clinit>
Aug 23, 2020 6:16:16 PM com.github.fommil.netlib.LAPACK <clinit>


lr: breeze.stats.regression.LeastSquaresRegressionResult = LeastSquaresRegressionResult(
  DenseVector(-1.6217143725490628E-7, -2.308730049264897E-5, 2.467183551996725E-5, -0.03254098698301093, 0.16172700628936792, 0.8381915917175606, -0.6156056910788755, -0.17473768951260324, -0.5943403814753612, -0.20105200928055197, 2.7639446316221324E-4, 9.826747991331102E-4, 0.462012841282421, -0.6975405221897882, -0.6168208301907278, -0.7427169056262426, 0.10606857021735167, -0.08010921159788657, 0.5833464201231637, -0.9578852542776571, -1.5485997606093498, -1.0010745302779724, -1.1485845720049965, -0.289665174329425, -1.3678079631982163, -0.4108668824117405, -1.8189726274298732),
  3.189779553980565E7
)

In [12]:
println("r^2=" + lr.rSquared)

r^2=3.189779553980565E7


In [13]:
val y_hat = X_test * lr.coefficients

Aug 23, 2020 6:16:18 PM com.github.fommil.netlib.BLAS <clinit>
Aug 23, 2020 6:16:18 PM com.github.fommil.netlib.BLAS <clinit>


y_hat: DenseVector[Double] = DenseVector(-3.959624132081683, -3.6590457752954944, -4.881317492842158, 3.360844047995688, 4.938969831984364, 2.568233929985988, 7.129879817533134, 6.038693854526601, 5.6000449480576755, -0.3477775690447432, 8.472667325131688, -1.1019696723826868, 0.7529561078014013, 11.536487945412993, 7.6883950655201705, 1.036373260362889, -2.773403063983352, 4.372971241407426, 5.440460001321872, 0.7596637304802862, 1.1294295968514114, 1.3481056880966027, 8.363149904285027, 5.882679008386152, -2.1449779465507235, -3.416674492041567, 0.48352891081036076, -0.6540574071363081, 5.172945033850297, -4.146827626222815, 1.1650344171622353, -0.549643779758259, -3.0001576744868057, 6.818945495912874, 0.39288105281872854, -0.8907673815594204, -1.5544235258330994, 8.664810145240034, -1.5781039123964824, 4.960698872626105, -4.468508592169382, -1.3129322884738042, 11.284060305933448, 4.306747129180394, 13.924828232330205, -0.10413342062912911, 13.454381161380098, 0.5986793272640393, 2

In [14]:
val resid = y_test - y_hat

resid: DenseVector[Double] = DenseVector(3.959624132081683, 3.6590457752954944, 4.881317492842158, -3.360844047995688, -4.938969831984364, -2.568233929985988, -4.129879817533134, -6.038693854526601, -5.6000449480576755, 0.3477775690447432, -8.472667325131688, 1.1019696723826868, -0.7529561078014013, -9.536487945412993, -7.6883950655201705, -1.036373260362889, 2.773403063983352, -4.372971241407426, -5.440460001321872, -0.7596637304802862, -1.1294295968514114, -1.3481056880966027, -4.363149904285027, -3.8826790083861518, 2.1449779465507235, 3.416674492041567, -0.48352891081036076, 0.6540574071363081, -5.172945033850297, 4.146827626222815, -1.1650344171622353, 0.549643779758259, 3.0001576744868057, -5.818945495912874, -0.39288105281872854, 0.8907673815594204, 1.5544235258330994, -8.664810145240034, 1.5781039123964824, -4.960698872626105, 4.468508592169382, 2.312932288473804, -11.284060305933448, -3.306747129180394, -5.924828232330205, 0.10413342062912911, 16.5456188386199, -0.598679327264

In [15]:
val sum_of_squares = resid.map(x=>x*x).reduce( _ + _ )

sum_of_squares: Double = 4238699.492533666

In [16]:
scala.math.sqrt((sum_of_squares / y_test.length))

res15: Double = 22.749631328531155

**Визуализируем результат.**  
Рассмотрим предсказания в зависимости от одного из признаков - baseTime

In [17]:
val baseTime_feature = X_test(::, 9)
val target = Scatter(
  baseTime_feature.toScalaVector,
  y_test.toScalaVector,
  mode = ScatterMode(ScatterMode.Markers)
)

val squaresResult = Scatter(
  baseTime_feature.toScalaVector,
  (X_test * lr.coefficients).toScalaVector,
  mode = ScatterMode(ScatterMode.Markers)
)

val data = Seq(target, squaresResult)
val layout = Layout(
  title = "Linear Regression Results: Base time feature vs y_true and predicted (trace 0, trace 1)"
)

plot(data, layout)

baseTime_feature: DenseVector[Double] = DenseVector(64.0, 61.0, 68.0, 33.0, 25.0, 27.0, 9.0, 11.0, 12.0, 45.0, 5.0, 52.0, 39.0, 7.0, 14.0, 35.0, 58.0, 25.0, 18.0, 44.0, 42.0, 42.0, 12.0, 22.0, 55.0, 69.0, 42.0, 51.0, 20.0, 63.0, 37.0, 50.0, 67.0, 9.0, 38.0, 52.0, 47.0, 7.0, 55.0, 22.0, 46.0, 70.0, 21.0, 36.0, 22.0, 47.0, 2.0, 60.0, 44.0, 37.0, 0.0, 25.0, 54.0, 64.0, 16.0, 5.0, 38.0, 0.0, 35.0, 12.0, 17.0, 70.0, 12.0, 56.0, 50.0, 35.0, 22.0, 53.0, 14.0, 5.0, 2.0, 11.0, 2.0, 71.0, 22.0, 59.0, 65.0, 2.0, 49.0, 32.0, 67.0, 52.0, 30.0, 1.0, 15.0, 49.0, 63.0, 7.0, 37.0, 10.0, 17.0, 57.0, 65.0, 37.0, 39.0, 24.0, 22.0, 43.0, 42.0, 21.0, 47.0, 30.0, 33.0, 16.0, 20.0, 57.0, 16.0, 21.0, 27.0, 16.0, 72.0, 40.0, 26.0, 24.0, 63.0, 66.0, 38.0, 15.0, 63.0, 4.0, 69.0, 18.0, 43.0, 70.0, 12.0, 20.0, 9.0, 63.0, 21.0, 70.0, 29.0, 6.0, 6.0, 12.0, 44.0, 11.0, 34.0, 55.0, 60.0, 53.0, 16.0, 16.0, 49.0, 29.0, 22.0, 36.0, 4.0, 21.0, 54.0, 14.0, 11.0, 52.0, 56.0, 47.0, 63.0, 51.0, 29.0, 53.0, 30.0, 15.0, 34.0, 6.

Зависимость целевой переменной от предсказанных значений.

In [18]:
val lr_results_scatterplot = Scatter(
  y_test.toScalaVector,
  y_hat.toScalaVector,
  mode = ScatterMode(ScatterMode.Markers)
)

val data = Seq(lr_results_scatterplot)
val layout = Layout(
  title = "Scatterplot y_true vs y_hat"
)

plot(data, layout)

lr_results_scatterplot: Scatter = Scatter(
  Some(
    Doubles(
      Vector(
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        3.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        2.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        4.0,
        2.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        1.0,
        0.0,
...
data: Seq[Scatter] = List(
  Scatter(
    Some(
      Doubles(
        Vector(
          0.0,
          0.0,
          0.0,
          0.0,
          0.0,
          0.0,
          3.0,
          0.0,
          0.0,
          0.0,
          0.0,
          0.0,
          0.0,
          2.0,
          0.0,
          0.0,
          0.0,
          0.0,
          0.0,
          0.0,
          0.0,
          0.0,
          4.0,
          2.0,
          0.0,
         

### Реализация линейной регрессии руками
1. Реализуйте свою версию алгоритма линейной регрессии, используя только базовый функционал Breeze для работы с матрицами (без использования breeze.stats.regression или других библиотек, реализующих регрессию в готовом виде).

In [19]:
val lr_coefficients = pinv(X_train.t * X_train) * X_train.t * y_train 

lr_coefficients: DenseVector[Double] = DenseVector(-1.6171640358761357E-7, -2.3002027443632434E-5, 2.4569033218467675E-5, -0.03313417304107766, 0.1617631316428278, 1.4932634956369906, -1.2708144666852874, -0.1747228548424338, -1.2494449962539191, -0.20154179537944208, 2.704507714869509E-4, 9.858566437991188E-4, 0.4318869864204405, -0.43919724875763233, -0.22804816100225178, -0.42043407560008256, 0.16424533375107248, 0.19439169392292271, 0.8418586569345988, -0.636036507025696, -1.1406029137860072, -0.3883574827461127, -0.7188784787962397, 0.2560989759474886, -0.7893880282426543, 0.23929099118986946, -1.2835456002778511)

In [20]:
val y_hat = X_test * lr_coefficients

y_hat: DenseVector[Double] = DenseVector(-4.04290663591515, -3.6129407091430217, -5.054712903117196, 3.5352125425470384, 4.986912514590989, 2.551161553155005, 7.12128052872291, 6.028961965113622, 5.589974628592879, -0.41637765160813034, 8.6044170927205, -1.0570307171051623, 0.9007005176805843, 11.667460553971441, 6.947844313899455, 0.9603445530389432, -2.7436818601538953, 3.8725094649494958, 5.5541505351226865, 0.699696473202683, 1.071137323697593, 1.2893029679690406, 8.24622412888817, 6.050686582305382, -2.2514570487989896, -3.9383453596343494, 0.03859621178961603, -0.7173164621976107, 4.67513038104878, -4.345510307766703, 1.0377265456738813, -0.6113783253027649, -3.520701254878706, 6.862729862817668, 0.5232909777327112, -0.9545397050654629, -1.5141292636987824, 8.823538194566089, -1.8054682307017302, 5.128603731428956, -4.1252388475353765, -1.3930244267987484, 11.333444656274098, 4.2205838379289125, 13.974830565413411, -0.13121037579892203, 13.577499802905699, 0.15805884086486877, 1.

In [21]:
val residuals = y_test - y_hat

residuals: DenseVector[Double] = DenseVector(4.04290663591515, 3.6129407091430217, 5.054712903117196, -3.5352125425470384, -4.986912514590989, -2.551161553155005, -4.12128052872291, -6.028961965113622, -5.589974628592879, 0.41637765160813034, -8.6044170927205, 1.0570307171051623, -0.9007005176805843, -9.667460553971441, -6.947844313899455, -0.9603445530389432, 2.7436818601538953, -3.8725094649494958, -5.5541505351226865, -0.699696473202683, -1.071137323697593, -1.2893029679690406, -4.246224128888169, -4.050686582305382, 2.2514570487989896, 3.9383453596343494, -0.03859621178961603, 0.7173164621976107, -4.67513038104878, 4.345510307766703, -1.0377265456738813, 0.6113783253027649, 3.520701254878706, -5.862729862817668, -0.5232909777327112, 0.9545397050654629, 1.5141292636987824, -8.823538194566089, 1.8054682307017302, -5.128603731428956, 4.1252388475353765, 2.3930244267987484, -11.333444656274098, -3.2205838379289125, -5.974830565413411, 0.13121037579892203, 16.4225001970943, -0.158058840

In [22]:
val sum_of_squares = residuals.map(x=>x*x).reduce( _ + _ )

sum_of_squares: Double = 4238611.171283844

In [23]:
scala.math.sqrt((sum_of_squares / y_test.length))

res22: Double = 22.749394311688803

Результат совпадает с предыдущим с точностью до 3 знака после запятой.